In [1]:
from google.colab import files
import numpy as np
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

Using TensorFlow backend.


In [3]:
files.upload()

{}

Open File and read contents

In [4]:
doc=open('part1.txt')
text=doc.read()
print(text[:200])

Harry Potter and the Sorcerer's Stone 

CHAPTER ONE 

THE BOY WHO LIVED 

Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say that they were perfectly normal, thank you very much. Th


Data Preprocessing and Cleaning

In [0]:
import string
tokens=text.split()
table = str.maketrans('', '', string.punctuation)
tokens = [w.translate(table) for w in tokens]
tokens = [word for word in tokens if word.isalpha()]
tokens = [word.lower() for word in tokens]

In [0]:
total_len=50+1
sequence_list=list()
for i in range(total_len,len(tokens)):
  seq=tokens[i-total_len:i]
  line=' '.join(seq)
  sequence_list.append(line)

Fit all text in tokenizer for using word embeddings

In [0]:
from keras.preprocessing.text import Tokenizer
tokenizer=Tokenizer()
tokenizer.fit_on_texts(sequence_list)
sequences=tokenizer.texts_to_sequences(sequence_list)

In [0]:
vocab_size=len(tokenizer.word_index)+1 

Make X and Y to fit into Model

In [0]:
sequences=np.array(sequences)
X=sequences[:,:-1]
Y=sequences[:,-1]
Y=to_categorical(Y,num_classes=vocab_size)

In [10]:
seq_length=X.shape[1]
print(seq_length)

50


Define Model

In [11]:
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(250, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(250, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 50)            296950    
_________________________________________________________________
lstm_1 (LSTM)                (None, 50, 250)           301000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               140400    
_________________________________________________________________
dense_1 (Dense)              (None, 250)               25250     
_________________________________________________________________
dense_2 (Dense)              (None, 5939)              1490689   
Total params: 2,254,289
Trainable params: 2,254,289
Non-trainable params: 0
_________________________________________________________________
None


In [12]:
model.fit(X, Y, batch_size=128, epochs=50)
model.save('model.h5')

Epoch 1/50
75088/75088 [==============================] - 145s 2ms/step - loss: 6.7164 - acc: 0.0466
Epoch 2/50
75088/75088 [==============================] - 144s 2ms/step - loss: 6.3483 - acc: 0.0513
Epoch 3/50
75088/75088 [==============================] - 146s 2ms/step - loss: 6.1545 - acc: 0.0621
Epoch 4/50
75088/75088 [==============================] - 146s 2ms/step - loss: 6.0114 - acc: 0.0680
Epoch 5/50
75088/75088 [==============================] - 145s 2ms/step - loss: 5.8826 - acc: 0.0779
Epoch 6/50
75088/75088 [==============================] - 144s 2ms/step - loss: 5.7587 - acc: 0.0866
Epoch 7/50
75088/75088 [==============================] - 144s 2ms/step - loss: 5.6511 - acc: 0.0953
Epoch 8/50
75088/75088 [==============================] - 142s 2ms/step - loss: 5.5616 - acc: 0.0994
Epoch 9/50
75088/75088 [==============================] - 143s 2ms/step - loss: 5.4602 - acc: 0.1041
Epoch 10/50
75088/75088 [==============================] - 143s 2ms/step - loss: 5.3875 - a

In [13]:
model.fit(X, Y, batch_size=128, epochs=50)
model.save('model2.h5')

Epoch 1/50
75088/75088 [==============================] - 147s 2ms/step - loss: 3.3682 - acc: 0.2711
Epoch 2/50
75088/75088 [==============================] - 147s 2ms/step - loss: 3.3273 - acc: 0.2759
Epoch 3/50
75088/75088 [==============================] - 147s 2ms/step - loss: 3.2861 - acc: 0.2822
Epoch 4/50
75088/75088 [==============================] - 147s 2ms/step - loss: 3.2468 - acc: 0.2886
Epoch 5/50
75088/75088 [==============================] - 148s 2ms/step - loss: 3.2054 - acc: 0.2945
Epoch 6/50
75088/75088 [==============================] - 147s 2ms/step - loss: 3.1650 - acc: 0.3002
Epoch 7/50
75088/75088 [==============================] - 146s 2ms/step - loss: 3.1188 - acc: 0.3087
Epoch 8/50
75088/75088 [==============================] - 147s 2ms/step - loss: 3.0853 - acc: 0.3130
Epoch 9/50
75088/75088 [==============================] - 147s 2ms/step - loss: 3.0406 - acc: 0.3191
Epoch 10/50
75088/75088 [==============================] - 147s 2ms/step - loss: 2.9937 - a

In [0]:
files.download('model.h5')

Predicting Text

In [0]:
from keras.preprocessing.sequence import pad_sequences
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
  result = list()
  in_text = seed_text
  for _ in range(n_words):
    encoded = tokenizer.texts_to_sequences([in_text])[0]
    encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
    yhat = model.predict_classes(encoded, verbose=0)
    out_word = ''
    for word, index in tokenizer.word_index.items():
      if index == yhat:
        out_word = word
        break
    in_text += ' ' + out_word
    result.append(out_word)
  return ' '.join(result)

In [26]:
from random import randint
seed_text = sequence_list[randint(0,len(sequence_list))]
print(seed_text)
 
# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)
print(generated)

its eyes to the ceiling it gave harry a look that said quite plainly i get that all the time i know harry murmured through the glass though he wasnt sure the snake could hear him it must be really annoying the snake nodded vigorously where do you come from anyway
well dumbledores only nicolas flamel we believed to cry the same dream you didnt have scars burning at his side malfoy looked at it and said harry quirrell telling him i dont understand why i thought you werent going to be changing them around the potters youd better let you


Final prediction<br>

its eyes to the ceiling it gave harry a look that said quite plainly i get that all the time i know harry murmured through the glass though he wasnt sure the snake could hear him it must be really annoying the snake nodded vigorously where do you come from anyway
well dumbledores only nicolas flamel we believed to cry the same dream you didnt have scars burning at his side malfoy looked at it and said harry quirrell telling him i dont understand why i thought you werent going to be changing them around the potters youd better let you